![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3.5: Object Oriented Programming
#### Written by Stevo Bailey ([stevo@berkeley.edu](mailto:stevo@berkeley.edu))

## Table of Contents

**[Object Oriented Programming Introduction](#intro)**

**[Chisel Supers](#super)**
1. [Module](#module)
1. [Bundle](#bundle)

# Object Oriented Programming Introduction<a name="intro"></a>
Scala and Chisel are object-oriented programming languages, meaning code may be compartmentalized into objects. Scala, which is built on Java, inherits many of Java's object-oriented features. However, as we'll see below, there are some differences. Chisel's hardware modules are similar to Verilog's modules, in that they can be instantiated and wired up as single or multiple instances.

# Chisel Supers<a name="super"></a>
You've seen `Module`s and `Bundle`s before, but it's important to realize that Chisel `Module`s are the superclass of all hardware blocks you will design, `Bundles` are the superclass of IO, and `Data` is the supertype of all Chisel types. Next we'll explore using these supers to create hierarchical hardware blocks and explore object reuse. You'll learn more about types and `Data` in the next Module.

## Module<a name="module"></a>
Whenever you want to create a hardware object in Chisel, it needs to have `Module` as a superclass. It's recommended that if you want to extend an existing `Module` that you create an abstract super version. Below is an example of creating a `Module` and connecting multiple instantiations of them together hierarchically.

In [ ]:
// Run this boilerplate for the necessary imports
import $ivy.`edu.berkeley.cs::chisel3:3.0.0`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1.0`
import $ivy.`edu.berkeley.cs::firrtl:1.0.0`
import chisel3._
import chisel3.util._
import chisel3.iotesters._
// withClock
import chisel3.experimental._


// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module.
// Don't worry about the implementation of this.
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

We'll create a hardware Gray encoder/decoder. The encode or decode operation choice is hardware programmable.

In [ ]:
import scala.math.pow

// create a module
class GrayCoder(bitwidth: Int) extends Module {
  val io = IO(new Bundle{
    val in = Input(UInt(bitwidth.W))
    val out = Output(UInt(bitwidth.W))
    val encode = Input(Bool()) // decode on false
  })
  
  when (io.encode) { //encode
    io.out := io.in ^ (io.in >> 1.U)
  } .otherwise { // decode, much more complicated
    io.out := Seq.fill(log2Ceil(bitwidth))(Wire(UInt(bitwidth.W))).zipWithIndex.fold((io.in, 0)){
      case ((w1: UInt, i1: Int), (w2: UInt, i2: Int)) => {
        w2 := w1 ^ (w1 >> pow(2, log2Ceil(bitwidth)-i2-1).toInt.U)
        (w2, i1)
      }
    }._1
  }
}

Give it a whirl!

In [ ]:
// test our gray coder
val bitwidth = 4
Driver(() => new GrayCoder(bitwidth)) {
  c => new PeekPokeTester(c) {
  
    def toBinary(i: Int, digits: Int = 8) =
      String.format("%" + digits + "s", i.toBinaryString).replace(' ', '0')

    println("Encoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
      poke(c.io.in, i)
      poke(c.io.encode, true)
      step(1)
      println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(peek(c.io.out).toInt, bitwidth)}")
    }
    
    println("Decoding:")
    for (i <- 0 until pow(2, bitwidth).toInt) {
      poke(c.io.in, i)
      poke(c.io.encode, false)
      step(1)
      println(s"In = ${toBinary(i, bitwidth)}, Out = ${toBinary(peek(c.io.out).toInt, bitwidth)}")
    }
  }
}

Gray codes are often used in asynchronous interfaces. Usually Gray counters are used rather than fully-featured encoders/decoders, but we'll the above module to simplify things. Below is an example AsyncFIFO, built using the above Gray coder. The control logic and tester is left as an exercise for later on. For now, look at how the Gray coder is instantiated multiple times and connected.

In [ ]:
class AsyncFIFO(depth: Int = 16) extends Module {
  val io = IO(new Bundle{
    // write inputs
    val write_clock = Input(Clock())
    val write_enable = Input(Bool())
    val write_data = Input(UInt(32.W))
    
    // read inputs/outputs
    val read_clock = Input(Clock())
    val read_enable = Input(Bool())
    val read_data = Output(UInt(32.W))
    
    // FIFO status
    val full = Output(Bool())
    val empty = Output(Bool())
  })
  
  // add extra bit to counter to check for fully/empty status
  assert(isPow2(depth), "AsyncFIFO needs a power-of-two depth!")
  val write_counter = withClock(io.write_clock) { Counter(io.write_enable && !io.full, depth*2)._1 }
  val read_counter = withClock(io.read_clock) { Counter(io.read_enable && !io.empty, depth*2)._1 }
  
  // encode
  val encoder = new GrayCoder(write_counter.getWidth)
  encoder.io.in := write_counter
  encoder.io.encode := true.B
  
  // synchronize
  val sync = withClock(io.read_clock) { ShiftRegister(encoder.io.out, 2) }
  
  // decode
  val decoder = new GrayCoder(read_counter.getWidth)
  decoder.io.in := sync
  decoder.io.encode := false.B
  
  // status logic goes here
  
}